# SP 500 Top 50 shares analysis

This file contains two sections. The first section will create a top 50 shares to buy based on their 1 year return. The second section will select the top 50 stocks based on their consistency in yields using hqm score.

Both sections use pretty much the same imports, which are covered below:

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import pprint
from chunks import chunks
from portfolioTools import portfolio_input
from scipy.stats import percentileofscore as score
from statistics import mean
from excelFormats import string_template_props, dollar_template_props, integer_template_props, percent_template_props

pp = pprint.PrettyPrinter(indent=2)

Importing sp 500 stocks. While it's best practice to use dynamic data for the index, a static list is enough for the purposes of this notebook.

In [2]:
stocks = pd.read_csv('../data/sp_500_stocks.csv')

### Secrets

The data source IEX cloud requires an api token even for the sandboxed data. `secrets.py` file is excluded from the repository and will require the user to get their own token and savei it in the mentioned file with the variable name imported below. 

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Method 1 - SP 500 Top 50 based on 1 year yield

This method will only use the 1 year change in value to determine the top 50 stocks to invest in.

### Executing the batch call

The following cells will retrieve SP 500 index from IEX. The api from IEX does not allow more than 100 tickers per call. Method chunks is used to split the list of symbols into 100-item sections.

In [4]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_csv_list = []

for i in range(0, len(symbol_groups)):
    symbol_csv_list.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [5]:
final_dataframe = pd.DataFrame(columns = my_columns)
# base url could be moved to a config file if similar properties increase in number 
BASE_URL = 'https://sandbox.iexapis.com/stable'
TYPES = 'stats,price'

for symbol_csv in symbol_csv_list:
    queries = f'symbols={symbol_csv}&types={TYPES}&token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url = f'{BASE_URL}/stock/market/batch?{queries}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_csv.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, # Ticker
                    data[symbol]['price'], # Price
                    data[symbol]['stats']['year1ChangePercent'], # One-Year Price Return
                    'N/A' # Number of Shares to Buy - This value will be amended later
                ],
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,125.230,0.794401,N/A
1,AAL,23.238,0.48126,N/A
2,AAP,196.100,1.108373,N/A
3,AAPL,125.440,1.01725,N/A
4,ABBV,109.670,0.631137,N/A
...,...,...,...,...
500,YUM,110.220,0.51494,N/A
501,ZBH,165.112,0.6741,N/A
502,ZBRA,475.866,1.48092,N/A
503,ZION,57.479,1.232179,N/A


### Removing low momentum stocks

The following cells select the top 50 highest performing stocks per 1 year yield. Pandas issues a warning in this section but the warning is irrelevant for our purposes.

In [6]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,VIAC,59.760,3.69509,N/A
1,LB,62.700,3.368322,N/A
2,FCX,33.700,3.32573,N/A
3,URI,331.810,2.45934,N/A
4,DVN,22.851,2.364958,N/A
5,KSS,61.120,2.289874,N/A
6,GPS,30.240,2.279992,N/A
7,FANG,77.450,2.238877,N/A
8,HAL,22.210,2.205881,N/A
9,NWSA,26.552,2.096478,N/A


### Calculating the number of shares to buy

Value input is required from the user to determine the number of shares to buy from each stock. Note that this input only affects the first method in this file. The second method has its own input prompt.

In [7]:
portfolio_size = portfolio_input()
portfolio_size

Enter the size of your portfolio:10000000


10000000.0

In [8]:
position_size = portfolio_size/len(final_dataframe)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,VIAC,59.760,3.69509,3346
1,LB,62.700,3.368322,3189
2,FCX,33.700,3.32573,5934
3,URI,331.810,2.45934,602
4,DVN,22.851,2.364958,8752
5,KSS,61.120,2.289874,3272
6,GPS,30.240,2.279992,6613
7,FANG,77.450,2.238877,2582
8,HAL,22.210,2.205881,9004
9,NWSA,26.552,2.096478,7532


## Method 2 - SP 500 top 50 based on consistent returns over 1, 3, 6, 12 months

This method will use the mean value of the 1, 3, 6, 12 month price changes. This is a very simplified analysis for elimininating stocks that may have had price changes due to one-off good news; the idea is that the rare good news that the stock has received will not persist for the entirety of the observed data points.

In [12]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score',
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_csv in symbol_csv_list:
    queries = f'symbols={symbol_csv}&types={TYPES}&token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url = f'{BASE_URL}/stock/market/batch?{queries}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_csv.split(','):
        hqm_dataframe = hqm_dataframe.append(
            # Note that N/A values will be amended in the following cells
            pd.Series(
                [
                    symbol, # Ticker
                    data[symbol]['price'], # Price
                    'N/A', # Number of Shares to Buy
                    data[symbol]['stats']['year1ChangePercent'], # One-Year Price Return
                    'N/A', # One-Year Return Percentile
                    data[symbol]['stats']['month6ChangePercent'], # Six-Month Price Return
                    'N/A', # Six-Month Return Percentile
                    data[symbol]['stats']['month3ChangePercent'], # Three-Month Price Return
                    'N/A', # Three-Month Return Percentile
                    data[symbol]['stats']['month1ChangePercent'], # One-Month Price Return
                    'N/A', # One-Month Return Percentile
                    'N/A', # HQM Score
                ],
                index = hqm_columns
            ),
            ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.600,N/A,0.818104,N/A,0.254167,N/A,0.042127,N/A,-0.001648,N/A,N/A
1,AAL,23.610,N/A,0.493259,N/A,0.853068,N/A,0.472764,N/A,0.093861,N/A,N/A
2,AAP,195.933,N/A,1.087578,N/A,0.248017,N/A,0.160477,N/A,0.159601,N/A,N/A
3,AAPL,122.290,N/A,0.996042,N/A,0.078342,N/A,-0.08855,N/A,-0.003471,N/A,N/A
4,ABBV,106.970,N/A,0.628989,N/A,0.245203,N/A,0.018119,N/A,-0.033362,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.337,N/A,0.50876,N/A,0.191747,N/A,0.003383,N/A,0.026779,N/A,N/A
501,ZBH,163.367,N/A,0.665901,N/A,0.180199,N/A,0.059037,N/A,-0.041633,N/A,N/A
502,ZBRA,471.910,N/A,1.531569,N/A,0.827694,N/A,0.232894,N/A,-0.075907,N/A,N/A
503,ZION,57.270,N/A,1.25329,N/A,0.988581,N/A,0.282273,N/A,0.003249,N/A,N/A


### Calculating momentum percentiles

Momentum percentile is used to rank stocks in comparison with each other to see which performed better for the given period. The percentiles will later be used to calculate a mean to determine the stocks that performed the best overall.

In [13]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',
]

# This is to make sure that score method does not
# throw an error due to NoneType comparison
for row in hqm_dataframe.index:
    for time_period in time_periods:
        price_return = f'{time_period} Price Return'
        if hqm_dataframe.loc[row, price_return] == None:
            hqm_dataframe.loc[row, price_return] = 0
        
for row in hqm_dataframe.index:
    for time_period in time_periods:
        return_percentile = f'{time_period} Return Percentile'
        price_return = f'{time_period} Price Return'
        
        hqm_dataframe.loc[row, return_percentile] = score(
            hqm_dataframe[price_return], 
            hqm_dataframe.loc[row, price_return]
        )/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.600,N/A,0.818104,0.663366,0.254167,0.483168,0.042127,0.328713,-0.001648,0.261386,N/A
1,AAL,23.610,N/A,0.493259,0.328713,0.853068,0.912871,0.472764,0.978218,0.093861,0.807921,N/A
2,AAP,195.933,N/A,1.087578,0.809901,0.248017,0.475248,0.160477,0.661386,0.159601,0.970297,N/A
3,AAPL,122.290,N/A,0.996042,0.766337,0.078342,0.215842,-0.08855,0.053465,-0.003471,0.247525,N/A
4,ABBV,106.970,N/A,0.628989,0.477228,0.245203,0.465347,0.018119,0.255446,-0.033362,0.114851,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.337,N/A,0.50876,0.342574,0.191747,0.386139,0.003383,0.221782,0.026779,0.427723,N/A
501,ZBH,163.367,N/A,0.665901,0.50495,0.180199,0.372277,0.059037,0.378218,-0.041633,0.089109,N/A
502,ZBRA,471.910,N/A,1.531569,0.920792,0.827694,0.910891,0.232894,0.807921,-0.075907,0.045545,N/A
503,ZION,57.270,N/A,1.25329,0.873267,0.988581,0.960396,0.282273,0.875248,0.003249,0.287129,N/A


### Calculating the HQM Score 

We now use the momentum percentiles to determine the quality of each stock for the given price change data points. 

In [14]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.600,N/A,0.818104,0.663366,0.254167,0.483168,0.042127,0.328713,-0.001648,0.261386,0.434158
1,AAL,23.610,N/A,0.493259,0.328713,0.853068,0.912871,0.472764,0.978218,0.093861,0.807921,0.756931
2,AAP,195.933,N/A,1.087578,0.809901,0.248017,0.475248,0.160477,0.661386,0.159601,0.970297,0.729208
3,AAPL,122.290,N/A,0.996042,0.766337,0.078342,0.215842,-0.08855,0.053465,-0.003471,0.247525,0.320792
4,ABBV,106.970,N/A,0.628989,0.477228,0.245203,0.465347,0.018119,0.255446,-0.033362,0.114851,0.328218
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.337,N/A,0.50876,0.342574,0.191747,0.386139,0.003383,0.221782,0.026779,0.427723,0.344554
501,ZBH,163.367,N/A,0.665901,0.50495,0.180199,0.372277,0.059037,0.378218,-0.041633,0.089109,0.336139
502,ZBRA,471.910,N/A,1.531569,0.920792,0.827694,0.910891,0.232894,0.807921,-0.075907,0.045545,0.671287
503,ZION,57.270,N/A,1.25329,0.873267,0.988581,0.960396,0.282273,0.875248,0.003249,0.287129,0.74901


### Selecting the top 50 momentum scores

Finally, we select the top 50 best performing stocks with respect to their mean momentum

In [15]:
hqm_dataframe.sort_values('HQM Score', inplace = True, ascending = False)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DISCK,40.691,N/A,1.762414,0.954455,1.67675,0.994059,1.070322,1.0,0.177116,0.978218,0.981683
1,LB,64.690,N/A,3.348995,0.99604,0.977635,0.956436,0.537765,0.990099,0.140925,0.942574,0.971287
2,DISCA,46.140,N/A,1.748338,0.950495,1.774282,0.99604,1.038946,0.99802,0.12649,0.912871,0.964356
3,NWSA,26.815,N/A,2.062705,0.980198,0.896385,0.936634,0.484088,0.982178,0.140245,0.936634,0.958911
4,GPS,30.090,N/A,2.266075,0.988119,0.780288,0.90297,0.422948,0.966337,0.139485,0.934653,0.94802
5,HIG,70.590,N/A,1.1974,0.851485,0.960409,0.954455,0.450119,0.974257,0.347018,1.0,0.94505
6,GM,57.769,N/A,1.659808,0.942574,0.982936,0.958416,0.364427,0.932673,0.121189,0.90495,0.934653
7,EXPE,179.680,N/A,1.658943,0.940594,0.880269,0.926733,0.37574,0.938614,0.115669,0.893069,0.924752
8,MHK,204.600,N/A,1.376451,0.893069,1.051256,0.966337,0.386557,0.944554,0.115638,0.891089,0.923762
9,FANG,78.170,N/A,2.183811,0.986139,1.558842,0.984158,0.553521,0.992079,0.077724,0.720792,0.920792


### Calculating the number of shares to buy

User input is used to determine the number of stocks that the user needs to buy for an even buy for all stocks. Note that this input is only relevant for method 2. Method 1 has its own user input prompt.

In [16]:
portfolio_input()

Enter the size of your portfolio:10000000


10000000.0

In [17]:
position_size = portfolio_size/len(hqm_dataframe)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DISCK,40.691,4915,1.762414,0.954455,1.67675,0.994059,1.070322,1.0,0.177116,0.978218,0.981683
1,LB,64.690,3091,3.348995,0.99604,0.977635,0.956436,0.537765,0.990099,0.140925,0.942574,0.971287
2,DISCA,46.140,4334,1.748338,0.950495,1.774282,0.99604,1.038946,0.99802,0.12649,0.912871,0.964356
3,NWSA,26.815,7458,2.062705,0.980198,0.896385,0.936634,0.484088,0.982178,0.140245,0.936634,0.958911
4,GPS,30.090,6646,2.266075,0.988119,0.780288,0.90297,0.422948,0.966337,0.139485,0.934653,0.94802
5,HIG,70.590,2833,1.1974,0.851485,0.960409,0.954455,0.450119,0.974257,0.347018,1.0,0.94505
6,GM,57.769,3462,1.659808,0.942574,0.982936,0.958416,0.364427,0.932673,0.121189,0.90495,0.934653
7,EXPE,179.680,1113,1.658943,0.940594,0.880269,0.926733,0.37574,0.938614,0.115669,0.893069,0.924752
8,MHK,204.600,977,1.376451,0.893069,1.051256,0.966337,0.386557,0.944554,0.115638,0.891089,0.923762
9,FANG,78.170,2558,2.183811,0.986139,1.558842,0.984158,0.553521,0.992079,0.077724,0.720792,0.920792


### Formatting Excel output

It is advised to use the artifacts folder for any output from the notebook. This particular directory is mounted by docker to make it available for both the host and the container.

In [18]:
writer = pd.ExcelWriter('/opt/artifacts/momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

In [19]:
string_template = writer.book.add_format(string_template_props)
dollar_template = writer.book.add_format(dollar_template_props)
integer_template = writer.book.add_format(integer_template_props)
percent_template = writer.book.add_format(percent_template_props)

In [20]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template],
}

sheet = writer.sheets['Momentum Strategy'] 

for column in column_formats.keys():
    sheet.set_column(f'{column}:{column}', 25, column_formats[column][1])
    sheet.write(f'{column}1', column_formats[column][0])
    


### Saving the excel file


In [21]:
writer.save()